In [42]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import push_notebook, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, FactorRange, Whisker
from bokeh.models.glyphs import VArea
from bokeh.models.tools import HoverTool
import bokeh.palettes
from bokeh.transform import factor_cmap
import math 

import pandas as pd

df = pd.read_csv('Results/iperf_duration.csv')
df = df.replace(-1, 0)
print(df.to_string())


     runtime image_name ressource_limit  repeat  duration type         send     received                                           filepath
0    default      iperf         default       1        10  tcp  944190000.0  943633000.0  /opt/auswertung/Raw/bench_iperf_dauer/default/...
1    default      iperf         default       1        10  udp    1041990.0          0.0  /opt/auswertung/Raw/bench_iperf_dauer/default/...
2    default      iperf         default      10        10  tcp  943644000.0  943562000.0  /opt/auswertung/Raw/bench_iperf_dauer/default/...
3    default      iperf         default      10        10  udp    1042010.0          0.0  /opt/auswertung/Raw/bench_iperf_dauer/default/...
4    default      iperf         default      11        10  tcp  943602000.0  943598000.0  /opt/auswertung/Raw/bench_iperf_dauer/default/...
5    default      iperf         default      11        10  udp    1041990.0          0.0  /opt/auswertung/Raw/bench_iperf_dauer/default/...
6    default      ip

In [43]:
grouped = df
grouped['send_mb'] = 0
grouped['re_mb'] = 0

for index, row in grouped.iterrows():
    grouped.loc[index, 'send_mb'] = float(row['send'])  / 1024 / 1024
    grouped.loc[index, 're_mb'] = float(row['received']) / 1024 / 1024
    
print(grouped.to_string())

     runtime image_name ressource_limit  repeat  duration type         send     received                                           filepath     send_mb       re_mb
0    default      iperf         default       1        10  tcp  944190000.0  943633000.0  /opt/auswertung/Raw/bench_iperf_dauer/default/...  900.449753  899.918556
1    default      iperf         default       1        10  udp    1041990.0          0.0  /opt/auswertung/Raw/bench_iperf_dauer/default/...    0.993719    0.000000
2    default      iperf         default      10        10  tcp  943644000.0  943562000.0  /opt/auswertung/Raw/bench_iperf_dauer/default/...  899.929047  899.850845
3    default      iperf         default      10        10  udp    1042010.0          0.0  /opt/auswertung/Raw/bench_iperf_dauer/default/...    0.993738    0.000000
4    default      iperf         default      11        10  tcp  943602000.0  943598000.0  /opt/auswertung/Raw/bench_iperf_dauer/default/...  899.888992  899.885178
5    default    

In [44]:

send_std = grouped.groupby(['duration', 'type'])['send_mb'].std().reset_index(name='send_std')
print(send_std)

re_std = grouped.groupby(['duration', 'type'])['re_mb'].std().reset_index(name='re_std')
print(re_std)

send_mean = grouped.groupby(['duration', 'type'])['send_mb'].mean().reset_index(name='send_mean')
print(send_mean)

re_mean = grouped.groupby(['duration', 'type'])['re_mb'].mean().reset_index(name='re_mean')
print(re_mean)


send_std['send_mean'] = send_mean[['send_mean']]
send_std['re_std'] = re_std[['re_std']]
send_std['send_mean'] = send_mean[['send_mean']]
send_std['re_mean'] = re_mean[['re_mean']]
grouped_std_mean = send_std
print(grouped_std_mean)

# 95% Vertrauensintervall
grouped_std_mean['send_abweichung_in_prozent'] = grouped_std_mean['send_std']  / math.sqrt(35) * 1.6909 * 100 / grouped_std_mean['send_mean']
grouped_std_mean['send_konf_intervall_unten'] = grouped_std_mean['send_mean'] - grouped_std_mean['send_std']  / math.sqrt(35) * 1.6909
grouped_std_mean['send_konf_intervall_oben'] = grouped_std_mean['send_mean'] + grouped_std_mean['send_std']  / math.sqrt(35) * 1.6909

grouped_std_mean['re_abweichung_in_prozent'] = grouped_std_mean['re_std']  / math.sqrt(35) * 1.6909 * 100 / grouped_std_mean['re_mean']
grouped_std_mean['re_konf_intervall_unten'] = grouped_std_mean['re_mean'] - grouped_std_mean['re_std']  / math.sqrt(35) * 1.6909
grouped_std_mean['re_konf_intervall_oben'] = grouped_std_mean['re_mean'] + grouped_std_mean['re_std']  / math.sqrt(35) * 1.6909

grouped_std_mean = grouped_std_mean.round(2)
print(grouped_std_mean)

#summe_delta_s_quadrat = filterd['delta_s_quadrat'].sum()
#standartabweichung = math.sqrt(summe_delta_s_quadrat/199)
#print('standartabweichung: ' + str(standartabweichung))

# Formel t Verteilung: http://www.statistics4u.info/fundstat_germ/cc_confidence_interval.html
# T Verteilungs tabelle: https://www.staff.uni-mainz.de/klenke/vorlesungen/vorl_ws19-2/tabellenwerk.pdf
# Nachschauen:
# n -1 = 39 <- richtig?
# a = 0,005 bei 99,5% Wahrscheinlichkeit
# Werte 38-> 2,712 40-> 2,704 ~ 39 -> 2,7105
# neu: 39 -> 2.7079
# Formel:
# Erwartungswert +- Standartabweichung / sqrt(40) * 2,7105

   duration type  send_std
0        10  tcp  0.561800
1        10  udp  0.000006
2        30  tcp  0.401908
3        30  udp  0.000000
4        60  tcp  0.053964
5        60  udp  0.000000
6       120  tcp  0.022298
7       120  udp  0.000000
8       240  tcp  0.622130
9       240  udp  0.000000
   duration type    re_std
0        10  tcp  0.771709
1        10  udp  0.000000
2        30  tcp  0.371063
3        30  udp  0.000000
4        60  tcp  0.025581
5        60  udp  0.000000
6       120  tcp  0.013678
7       120  udp  0.000000
8       240  tcp  0.622867
9       240  udp  0.000000
   duration type   send_mean
0        10  tcp  900.159427
1        10  udp    0.993726
2        30  tcp  898.497363
3        30  udp    0.997906
4        60  tcp  898.198237
5        60  udp    0.998955
6       120  tcp  897.980063
7       120  udp    0.999479
8       240  tcp  897.997693
9       240  udp    0.999737
   duration type     re_mean
0        10  tcp  899.735015
1        10  udp    0.000000


In [45]:
export_csv = grouped_std_mean.to_csv (r'Results/iperf_dauer.csv', index = None, header=True, decimal=',')

In [50]:
# tcp send

grouped_std_mean.sort_values(by=['duration'], inplace=True)
grouped_std_mean['duration']= grouped_std_mean['duration'].astype(str)

grouped_tcp = grouped_std_mean[(grouped_std_mean.type == 'tcp')]
source = ColumnDataSource(grouped_tcp)
duration =  list( dict.fromkeys(source.data['duration'].tolist()))
print(duration)

p = figure(
  x_range=duration,
  title = 'iperf Benchmark TCP Sendeleistung abhängig von der Messdauer',
  x_axis_label ='Dauer [s]',
  y_axis_label ='Übertragunsrate [MegaBytes pro Sekunde]',
  plot_width=800
)

color_map = factor_cmap(field_name='duration', palette=bokeh.palettes.d3['Category10'][5], factors=duration)

p.vbar(
    x='duration',
    top='send_mean',
    width=0.60,
    source=source,
    color=color_map
)

p.add_layout(
    Whisker(source=source, base="duration", upper="send_konf_intervall_oben", lower="send_konf_intervall_unten", level="overlay")
)

output_notebook()

show(p)












['10', '120', '240', '30', '60']


Loading BokehJS ...

In [55]:
# tcp recived
print(duration)

p = figure(
  x_range=duration,
  title = 'iperf Benchmark TCP Empfangsleistung abhängig von der Messdauer',
  x_axis_label ='Dauer [s]',
  y_axis_label ='Übertragunsrate [MegaBytes pro Sekunde]',
  width=400,
  height=400
)

color_map = factor_cmap(field_name='duration', palette=bokeh.palettes.d3['Category10'][5], factors=duration)

p.vbar(
    x='duration',
    top='re_mean',
    width=0.60,
    source=source,
    color=color_map
)

#p.add_layout(
#    Whisker(source=source, base="duration", upper="re_konf_intervall_oben", lower="re_konf_intervall_unten", level="overlay")
#)

output_notebook()

show(p)

['10', '120', '240', '30', '60']


Loading BokehJS ...

In [48]:
# udp send

grouped_tcp = grouped_std_mean[(grouped_std_mean.type == 'udp')]
source = ColumnDataSource(grouped_tcp)
duration =  list( dict.fromkeys(source.data['duration'].tolist()))
print(duration)

p = figure(
  x_range=duration,
  title = 'iperf Benchmark Ergebnisse abhängig von der Messdauer',
  x_axis_label ='Dauer',
  y_axis_label ='Requests pro Sekunde',
  plot_width=800
)

color_map = factor_cmap(field_name='duration', palette=bokeh.palettes.d3['Category10'][5], factors=duration)

p.vbar(
    x='duration',
    top='send_mean',
    width=0.60,
    source=source,
    color=color_map
)

p.add_layout(
    Whisker(source=source, base="duration", upper="send_konf_intervall_oben", lower="send_konf_intervall_unten", level="overlay")
)

output_notebook()

show(p)

['10', '30', '60', '120', '240']


Loading BokehJS ...